# CKP8366 - TÓPICOS AVANÇADOS - APRENDIZAGEM DE MÁQUINA PROBABILÍSTICA

<img  src="https://img.shields.io/badge/UFC_CKP8366-VAUX GOMES-000000?style=for-the-badge&logo=" /> <img src="https://img.shields.io/badge/Jupyter-000000?style=for-the-badge&logo=jupyter&logoColor=white" /> <img src="https://img.shields.io/badge/Python-000000?style=for-the-badge&logo=python&logoColor=white" />

In [2]:
import numpy as np
from scipy.stats import norm, multivariate_normal

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

## Probabilistic Principal Components Analysis (PPCA)

In [ ]:
class ProbabilisticPCA:
  def __init__(self, L, epochs=50, epsilon=0, seed=42):
    self.L = L
    self.epochs = epochs
    self.epsilon = epsilon
    self.seed = seed
    
  def fit(self, X, verbose=False):
    N, D = X.shape
    np.random.seed(self.seed)
    
    # Safety
    L = min(max(1, self.L), D)
    
    # Step 1.1
    mu = np.mean(X, axis=0)
    W = np.random.normal(0, 0.1, size=(D, L))
    sigma2 = np.var(X, axis=0).mean() + self.epsilon # Variância média
    
    #
    likelihoods = np.zeros(self.epochs)
    
    # Debug
    if verbose:
      print(mu.shape, W.shape, sigma2, self.epsilon)
    
    for e in range(self.epochs):
      # Step E / Step 2.1
      Minv = np.linalg.inv(W.T @ W + sigma2 + np.eye(L))
      MinvWT = Minv @ W.T
      
      # Debug
      if verbose and e == 0:
        print(Minv.shape, MinvWT.shape)

      Ez = np.empty((N, L))
      EzzT = np.empty((N, L, L))
      
      for i in range(N):
        Ez[i] = MinvWT @ (X[i] - mu)
        EzzT[i] = (sigma2 * Minv) + Ez[i] @ Ez[i].T
        
      # Debug
      if verbose and e == 0:
        print(Ez.shape, EzzT.shape)
        
      # Step M / Step 2.2
      W = np.empty((N, D, L))
      sigma2 = self.epsilon
      
      for i in range(N):
        W[i] = np.outer((X[i] - mu), Ez[i])
        
      W = np.sum(W, axis=0) @ np.linalg.inv(np.sum(EzzT, axis=0))
      
      #
      sigma2 = 2
      
      for i in range(N):

#### Carregamento dos dados

In [280]:
# Data
X = np.genfromtxt('./files/mnist_5.csv', delimiter=',')

# Normalization MinMax Keeping the only zeros columns
min_ = X.min(axis=0)
max_ = X.max(axis=0)
mask = min_ != max_

X[:, mask] = (X[:, mask] - min_[mask]) / (max_[mask] - min_[mask])

# X = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0)) # Normalization 0 ~ 1
X.shape

(6313, 784)

In [268]:
# init
L = 5

# fit
N, D = X.shape
L = min(max(1, L), D)
likelihood = []

# Step 1
mu = np.mean(X, axis=0)
W = np.random.normal(0, 0.1, size=(D, L))
sigma2 = np.var(X)

# Debug
print(mu.shape, W.shape, sigma2)

(784,) (784, 5) 6016.445442380658


In [277]:
# Step 2.1 (Expectation)
Minv = np.linalg.inv(W.T @ W + sigma2 * np.eye(L))
MinvWT = Minv @ W.T

# Debug
print(Minv.shape, MinvWT.shape)

Ez = np.empty((N, L))
EzzT = np.empty((N, L, L))

for i in range(N):
  Ez[i] = MinvWT @ (X[i] - mu)
  EzzT[i] = (sigma2 * Minv) + Ez[i] @ Ez[i].T
  
# Debug
print(Ez.shape, EzzT.shape)

# Step 2.2 (Maximization)
W = np.empty((N, D, L))

for i in range(N):
  W[i] = np.outer((X[i] - mu), Ez[i])

W = np.sum(W, axis=0) @ np.linalg.inv(np.sum(EzzT, axis=0))

# Ou
# W = np.einsum('ni,nj->ij', X - mu, Ez) @ np.linalg.inv(np.einsum('nij->ij', EzzT))

#
sigma2 = 0

for i in range(N):
  sigma2 += np.linalg.norm(X[i] - mu) - (2 * Ez[i].T @ W.T @ (X[i]- mu)) + np.linalg.trace(EzzT[i] @ W.T @ W)

sigma2 /= 1 / (N * D)

# Debug
print(W.shape, sigma2)


(5, 5) (5, 784)
(6313, 5) (6313, 5, 5)
(784, 5) 54566441886104.33


In [275]:
# Likelihood
epsilon = 1e-6
M = W.T @ W + (sigma2 + epsilon) * np.eye(L)

Sigma = W @ W.T + sigma2 * np.eye(D)
SigmaInv = 0.5 * sigma2 * np.eye(D) -0.5 * sigma2 * (W @ np.linalg.inv(M) @ W.T) # Usando identidade de Woodbury
logdet = 2 * np.sum(np.log(np.diag(np.linalg.cholesky(Sigma)))) # Usando Cholesky
 
part = 0

for i in range(N):
  diff = X[i] - mu
  part += diff @ SigmaInv @ diff
  
likelihood.append((-0.5 * (N * D) * np.log(2 * np.pi)) - (0.5 * N * logdet) - (0.5 * part))
likelihood[-1]

np.float64(-2.6637497359477565e+23)

In [276]:
likelihood

[np.float64(-1.1286202273565767e+23),
 np.float64(-2.6637497359477565e+23),
 np.float64(-2.6637497359477565e+23)]